# Chapter 11 - URLs and Multi-page Apps

* Getting to know the Location and Link components 
* Extracting and using attributes of URLs 
* Parsing URLs and using their components to modify parts of the app 
* Restructuring your app to cater for multiple layouts 
* Adding dynamically generated URLs to the app 
* Incorporating the new URL interactivity into the app 

In [1]:
from urllib.parse import parse_qs, unquote

import plotly
import plotly.express as px
import plotly.graph_objects as go
import dash
from dash import callback_context
import jupyter_dash as jd
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
import dash_bootstrap_components as dbc
from dash.dependencies import Output, Input, State, ALL, ALLSMALLER, MATCH
from dash.exceptions import PreventUpdate

from dash_table import DataTable
import pandas as pd
import numpy as np
pd.options.display.max_columns = None

for p in [plotly, dash, jd, dcc, html, dbc, pd, np]:
    print(f'{p.__name__:-<30}v{p.__version__}')
    

plotly------------------------v4.14.3
dash--------------------------v1.20.0
jupyter_dash------------------v0.4.0
dash_core_components----------v1.16.0
dash_html_components----------v1.1.3
dash_bootstrap_components-----v0.12.0
pandas------------------------v1.2.3
numpy-------------------------v1.20.1


In [2]:
poverty = pd.read_csv('../data/poverty.csv', low_memory=False)
poverty.head(2)

,Country Name,Country Code,year,"Annualized growth in per capita real survey mean consumption or income, bottom 40% (%)","Annualized growth in per capita real survey mean consumption or income, top 10% (%)","Annualized growth in per capita real survey mean consumption or income, top 60% (%)","Annualized growth in per capita real survey mean consumption or income, total population (%)",Annualized growth in per capita real survey median income or consumption expenditure (%),GINI index (World Bank estimate),Growth component of change in poverty at $1.90 a day (2011 PPP) (% of change),Growth component of change in poverty at $3.20 a day (2011 PPP) (% of change),Growth component of change in poverty at $5.50 a day (2011 PPP) (% of change),Income share held by fourth 20%,Income share held by highest 10%,Income share held by highest 20%,Income share held by lowest 10%,Income share held by lowest 20%,Income share held by second 20%,Income share held by third 20%,Median daily per capita income or consumption expenditure (2011 PPP),"Multidimensional poverty, Drinking water (% of population deprived)","Multidimensional poverty, Educational attainment (% of population deprived)","Multidimensional poverty, Educational enrollment (% of population deprived)","Multidimensional poverty, Electricity (% of population deprived)","Multidimensional poverty, Headcount ratio (% of population)","Multidimensional poverty, Monetary poverty (% of population deprived)","Multidimensional poverty, Sanitation (% of population deprived)",Number of poor at $1.90 a day (2011 PPP) (millions),Number of poor at $3.20 a day (2011 PPP) (millions),Number of poor at $5.50 a day (2011 PPP) (millions),"Population, total",Poverty gap at $1.90 a day (2011 PPP) (%),Poverty gap at $3.20 a day (2011 PPP) (% of population),Poverty gap at $5.50 a day (2011 PPP) (% of population),Poverty headcount ratio at $1.90 a day (2011 PPP) (% of population),"Poverty headcount ratio at $1.90 a day, Female (2011 PPP) (% of female population)","Poverty headcount ratio at $1.90 a day, Male (2011 PPP) (% of male population)","Poverty headcount ratio at $1.90 a day, age 0-14 (2011 PPP) (% of population age 0-14)","Poverty headcount ratio at $1.90 a day, age 15-64 (2011 PPP) (% of population age 15-64)","Poverty headcount ratio at $1.90 a day, age 65+ (2011 PPP) (% of population age 65+)","Poverty headcount ratio at $1.90 a day, rural (2011 PPP) (% of rural population)","Poverty headcount ratio at $1.90 a day, urban (2011 PPP) (% of urban population)","Poverty headcount ratio at $1.90 a day, with primary education (2011 PPP) (% of population age 16+ with primary education)","Poverty headcount ratio at $1.90 a day, with secondary education (2011 PPP) (% of population age 16+ with secondary education)","Poverty headcount ratio at $1.90 a day, without education (2011 PPP) (% of population age 16+ without education)","Poverty headcount ratio at $1.90 a day, with Tertiary/post-secondary education (2011 PPP) (% of population age 16+ with Tertiary/post-secondary education)",Poverty headcount ratio at $3.20 a day (2011 PPP) (% of population),Poverty headcount ratio at $5.50 a day (2011 PPP) (% of population),Poverty headcount ratio at national poverty lines (% of population),"Poverty headcount ratio at national poverty lines (% of population), including noncomparable values",Redistribution component of change in poverty at $1.90 a day (2011 PPP) (% of change),Redistribution component of change in poverty at $3.20 a day (2011 PPP) (% of change),Redistribution component of change in poverty at $5.50 a day (2011 PPP) (% of change),"Survey mean consumption or income per capita, bottom 40% (2011 PPP $ per day)","Survey mean consumption or income per capita, top 10% (2011 PPP $ per day)","Survey mean consumption or income per capita, top 60% (2011 PPP $ per day)","Survey mean consumption or income per capita, total population (2011 PPP $ per day)",Short Name,Table Name,Long Name,2-alpha code,Currency Unit,Spe

In [3]:
series = pd.read_csv('../data/PovStatsSeries.csv')
series.head(2)

,Series Code,Topic,Indicator Name,Short definition,Long definition,Unit of measure,Periodicity,Base Period,Other notes,Aggregation method,Limitations and exceptions,Notes from original source,General comments,Source,Statistical concept and methodology,Development relevance,Related source links,Other web links,Related indicators,License Type,Unnamed: 20
0,SI.DST.02ND.20,Poverty: Income distribution,Income share held by second 20%,NaN,Percentage share of income or consumption is t...,%,Annual,NaN,NaN,NaN,"Despite progress in the last decade, the chall...",NaN,The World Bank’s internationally comparable po...,"World Bank, Development Research Group. Data a...",Inequality in the distribution of income is re...,The World Bank Group's goal of promoting share...,NaN,NaN,NaN,CC BY-4.0,NaN
1,SI.DST.03RD.20,Poverty: Income distribution,Income share held by third 20%,NaN,Percentage share of income or consumption is t...,%,Annual,NaN,NaN,NaN,"Despite progress in the last decade, the chall...",NaN,The World Bank’s internationally comparable po...,"World Bank, Development Research Group. Data a...",Inequality in the distribution of income is re...,The World Bank Group's goal of promoting share...,NaN,NaN,NaN,CC BY-4.0,NaN


In [4]:
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.COSMO])

app.layout = html.Div([
    dcc.Location(id='location'),
    html.Div(id='output')
])

@app.callback(Output('output', 'children'),
              Input('location', 'href'))
def display_href(href):
    return f"You are at: {href}."

app.run_server(mode='inline')

In [5]:
app = JupyterDash(__name__,
                  external_stylesheets=[dbc.themes.COSMO])

app.layout = html.Div([
    dbc.Row([
        dbc.Col(lg=1),
        dbc.Col([
            html.Br(),
            dcc.Location(id='location'),
            html.A(href='/path',
                   children='Go to a direcotory path'),
            html.Br(),
            dcc.Link(href='/path/search?one=1&two=2',
                     children='Go to search page'),
            html.Br(),
            dcc.Link(href='path/?hello=HELLO#hash_string',
                     children='Go to a page with a hash'),
            html.Div(id='output')
            ])
        ])
    ])

@app.callback(Output('output', 'children'),
              Input('location', 'pathname'),
              Input('location', 'search'),
              Input('location', 'href'),
              Input('location', 'hash'))
def show_url_parts(pathname, search, href, hash_):
    return html.Div([
        html.Br(), html.Br(), 
        f"href: {href}",
        html.Br(),
        f"path: {pathname}",
        html.Br(),
        f"search: {search}",
        html.Br(),
        f"hash: {hash_}",
        ])

app.run_server(port=8054)

Dash app running on http://127.0.0.1:8054/


In [6]:
parse_qs('1=one&2=two&20=twenty')

{'1': ['one'], '2': ['two'], '20': ['twenty']}

In [7]:
parse_qs('country_code=CAN&year=2020&inidcator=SI.DST.02ND.20')

{'country_code': ['CAN'], 'year': ['2020'], 'inidcator': ['SI.DST.02ND.20']}

In [8]:
unquote('Bosnia%20and%20Herzegovina')

'Bosnia and Herzegovina'

In [9]:
unquote('Congo,%20Dem.%20Rep.')

'Congo, Dem. Rep.'

In [10]:
country_df = pd.read_csv('../data/PovStatsCountry.csv').drop(['Unnamed: 30'], axis=1)
country_df[country_df['Short Name'].str.contains(unquote('Congo'))]

,Country Code,Short Name,Table Name,Long Name,2-alpha code,Currency Unit,Special Notes,Region,Income Group,WB-2 code,National accounts base year,National accounts reference year,SNA price valuation,Lending category,Other groups,System of National Accounts,Alternative conversion factor,PPP survey year,Balance of Payments Manual in use,External debt Reporting status,System of trade,Government Accounting concept,IMF data dissemination standard,Latest population census,Latest household survey,Source of most recent Income and expenditure data,Vital registration complete,Latest agricultural census,Latest industrial data,Latest trade data
28,COD,Dem. Rep. Congo,"Congo, Dem. Rep.",Democratic Republic of the Congo,CD,Congolese franc,NaN,Sub-Saharan Africa,Low income,ZR,2005,NaN,Value added at basic prices (VAB),IDA,HIPC,Country uses the 1993 System of National Accou...,1999–2001,2011,BPM6,Actual,Special trade system,Budgetary central government,Enhanced General Data Dissemination System (e-...,1984,"Multiple Indicator Cluster Survey, 2017/18","1-2-3 survey (1-2-3), 2012/13",NaN,NaN,NaN,2016.0
29,COG,Congo,"Congo, Rep.",Republic of Congo,CG,Central African CFA franc,NaN,Sub-Saharan Africa,Lower middle income,CG,1990,NaN,Value added at producer prices (VAP),Blend,HIPC,Country uses the 1968 System of National Accou...,1993,2011,BPM6,Actual,Special trade system,Budgetary central government,Enhanced General Data Dissemination System (e-...,2007,Multiple Indicator Cluster Survey 2015/16,Core Welfare Indicator Questionnaire Survey (C...,NaN,2014-2015,2009.0,2017.0


In [11]:
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.COSMO])

countries = poverty[poverty['is_country']]['Country Name'].drop_duplicates().sort_values().tolist()


main_layout = html.Div([
    html.Div([
    dbc.NavbarSimple([
            dbc.DropdownMenu([
                dbc.DropdownMenuItem(country, href=country) for country in countries
            ], label='Select country'),
    ], brand='Home',brand_href='/', light=True),
    dbc.Row([
        dbc.Col(lg=1, md=1, sm=1),
        dbc.Col([
            
            dcc.Location(id='location'),
            html.Div(id='main_content')
        ], lg=10),
    ])
], style={'backgroundColor': '#E5ECF6'})
])

country_dashboard = html.Div([
    html.Br(),
        html.H1(id='country_heading'),
        dbc.Row([
            dbc.Col(dcc.Graph(id='country_page_graph'))
        ]),
        dbc.Row([
            dbc.Col([
                dbc.Label('Select indicator:'),
                dcc.Dropdown(id='country_page_indicator_dropdown',
                             placeholder='Choose an indicator',
                             value='Population, total',
                             options=[{'label': indicator, 'value': indicator}
                                     for indicator in poverty.columns[3:54]]),                
            ]),
            dbc.Col([
                dbc.Label('Select countries:'),
                dcc.Dropdown(id='country_page_contry_dropdown',
                             placeholder='Select one or more countries to compare',
                             multi=True,
                             options=[{'label': c, 'value': c}
                                       for c in countries]),
            ])
        ]),
        html.Br(), html.Br(),
        html.Div(id='country_table')
])

indicators_dashboard = html.Div([
    html.H1("Indicators Dashboard")
])

app.validation_layout = html.Div([
    main_layout,
    indicators_dashboard,
    country_dashboard,
])

app.layout = main_layout


@app.callback(Output('main_content', 'children'),
              Input('location', 'pathname'))
def display_content(pathname):
    if unquote(pathname[1:]) in countries:
        return country_dashboard
    else:
        return indicators_dashboard

@app.callback(Output('country_page_contry_dropdown', 'value'),
              Input('location', 'pathname'))
def set_dropdown_values(pathname):
    if unquote(pathname[1:]) in countries:
        country = unquote(pathname[1:])
        return [country]

@app.callback(Output('country_heading', 'children'),
              Output('country_page_graph', 'figure'),
              Output('country_table', 'children'),
              Input('location', 'pathname'),
              Input('country_page_contry_dropdown', 'value'),
              Input('country_page_indicator_dropdown', 'value'))
def plot_country_charts(pathname, countries, indicator):
    if (not countries) or (not indicator):
        raise PreventUpdate
    if unquote(pathname[1:]) in countries:
        country = unquote(pathname[1:])
    df = poverty[poverty['is_country'] & poverty['Country Name'].isin(countries)]
    fig = px.line(df,
                  x='year',
                  y=indicator,
                  title='<b>' + indicator + '</b><br>' + ', '.join(countries),
                  color='Country Name')
    fig.layout.paper_bgcolor = '#E5ECF6'
    table = country_df[country_df['Short Name'] == countries[0]].T.reset_index()
    if table.shape[1] == 2:
        table.columns = [countries[0] + ' Info', '']
        table = dbc.Table.from_dataframe(table)
    else:
        table = html.Div()
    return country + ' Poverty Data', fig, table


app.run_server(port=8181)

Dash app running on http://127.0.0.1:8181/
